In [8]:
import pandas as pd
import warnings
from itertools import combinations, permutations

In [9]:
excel_sheet = pd.read_excel("../../Data/milk_plasma_all_data_mrg_020821_v3.xlsx", sheet_name=[0, 1, 2])
full_sheet = excel_sheet[0].copy()
neutral_sheet = excel_sheet[1].copy()
ionizable_sheet = excel_sheet[2].copy()
ionizable_full_sheet: pd.DataFrame = full_sheet.loc[full_sheet["CHARGED"] == 1].copy()
neutral_full_sheet: pd.DataFrame = full_sheet.loc[full_sheet["neutral"] == 1].copy()


# df['your_column_name'].isin(["Log_MP_RATIO"]).value_counts()


df_dict: dict = {"full": full_sheet,
                 "neutral": neutral_sheet,
                 "ionizable": ionizable_sheet,
                 "charged verif":ionizable_full_sheet,
                 "neutral verif": neutral_full_sheet
                 }
for key, value in df_dict.items():
    print("====== ", key, " =======")
    print("\t>>> Before: ", value.shape)
    # Drop rows with NaN
    value.dropna(axis=0, inplace=True)
    # Drop non numerical cols
    non_numerical_cols_to_drop = [c for c in value.select_dtypes(exclude="number").columns if c != "set"]
    value.drop(columns=non_numerical_cols_to_drop, inplace=True)
    # Remove all estimated cols
    cols_to_remove: list = [c for c in value.columns if '$' in c]
    value.drop(columns=cols_to_remove, inplace=True)
    # Drop non informative column
    non_informative_cols = ["index", "smiles", "conc",   "name",   "CLASS", "ENV_CHEM",  "MP_RATIO", "FP:MACCS", "neutral",	"TEST",	"CHARGED", "group"]
    non_informative_cols = [c for c in value.columns if c in non_informative_cols]
    value.drop(columns=non_informative_cols, inplace=True)
    print("\t>>> After: ", value.shape)

# Checking we have the right amount of features
for key, value in df_dict.items():
    if value.shape[1] != 480:
        warnings.warn(key, " has not the right feature count\n\texpected: ", 480, "\n\tactual: ", value.shape[1])

======  full  =======
	>>> Before:  (184, 497)
	>>> After:  (183, 480)
======  neutral  =======
	>>> Before:  (111, 490)
	>>> After:  (111, 480)
======  ionizable  =======
	>>> Before:  (72, 490)
	>>> After:  (72, 480)
======  charged verif  =======
	>>> Before:  (72, 497)
	>>> After:  (72, 480)
======  neutral verif  =======
	>>> Before:  (111, 497)
	>>> After:  (111, 480)
======  full  =======
	>>> Before:  (184, 497)
	>>> After:  (183, 480)
======  neutral  =======
	>>> Before:  (111, 490)
	>>> After:  (111, 480)
======  ionizable  =======
	>>> Before:  (72, 490)
	>>> After:  (72, 480)
======  charged verif  =======
	>>> Before:  (72, 497)
	>>> After:  (72, 480)
======  neutral verif  =======
	>>> Before:  (111, 497)
	>>> After:  (111, 480)


In [10]:
comb_list = permutations(df_dict, 2)

for df_name in list(comb_list):
    correlation = df_dict[df_name[0]]["Log_MP_RATIO"].isin(df_dict[df_name[1]]["Log_MP_RATIO"]).value_counts().index.tolist()
    if len(correlation) > 1:
        print(df_name[0], "is partially in", df_name[1])
    elif True in correlation:
        print(df_name[0], " is fully in ", df_name[1])

full is partially in neutral
full is partially in ionizable
full is partially in charged verif
full is partially in neutral verif
neutral  is fully in  full
neutral is partially in ionizable
neutral is partially in charged verif
neutral  is fully in  neutral verif
ionizable  is fully in  full
ionizable is partially in neutral
ionizable  is fully in  charged verif
ionizable is partially in neutral verif
charged verif  is fully in  full
charged verif is partially in neutral
charged verif  is fully in  ionizable
charged verif is partially in neutral verif
neutral verif  is fully in  full
neutral verif  is fully in  neutral
neutral verif is partially in ionizable
neutral verif is partially in charged verif
full is partially in neutral
full is partially in ionizable
full is partially in charged verif
full is partially in neutral verif
neutral  is fully in  full
neutral is partially in ionizable
neutral is partially in charged verif
neutral  is fully in  neutral verif
ionizable  is fully in 

In [11]:
from qsar.preprocessing.feature_selector import FeatureSelector

FS_full = FeatureSelector(df_dict["full"], cols_to_ignore=["set"]).transform()
FS_ionizable = FeatureSelector(df_dict["ionizable"], cols_to_ignore=["set"]).transform()
FS_neutral = FeatureSelector(df_dict["neutral"], cols_to_ignore=["set"]).transform()

print("Shape of:\n\tfull\t\t:\t", FS_full.shape,"\n\tionizable\t:\t", FS_ionizable.shape, "\n\tneutral\t\t:\t", FS_neutral.shape)

Shape of:
	full		:	 (183, 359) 
	ionizable	:	 (72, 345) 
	neutral		:	 (111, 333)
Shape of:
	full		:	 (183, 359) 
	ionizable	:	 (72, 345) 
	neutral		:	 (111, 333)


In [12]:
# Separating test and train
full_train: pd.DataFrame = FS_full.loc[FS_full["set"] == "TRAINING"].copy().drop(columns=["set"])
full_test: pd.DataFrame = FS_full.loc[FS_full["set"] == "TEST"].copy().drop(columns=["set"])

# Ionized
ionizable_train: pd.DataFrame = FS_ionizable.loc[FS_ionizable["set"] == "TRAINING"].copy().drop(columns=["set"])
ionizable_test: pd.DataFrame = FS_ionizable.loc[FS_ionizable["set"] == "TEST"].copy().drop(columns=["set"])

# Neutral
neutral_train: pd.DataFrame = FS_neutral.loc[FS_neutral["set"] == "TRAINING"].copy().drop(columns=["set"])
neutral_test: pd.DataFrame = FS_neutral.loc[FS_neutral["set"] == "TEST"].copy().drop(columns=["set"])


print("===== FULL =====\n\tTrain:\t", full_train.shape, "\n\tTest:\t", full_test.shape)
print("===== IONIZABLE =====\n\tTrain:\t", ionizable_train.shape, "\n\tTest:\t", ionizable_test.shape)
print("===== NEUTRAL =====\n\tTrain:\t", neutral_train.shape, "\n\tTest:\t", neutral_test.shape)

print("===== CHECK IF TESTS ARE IN OTHER DATAFRAMES =====")
print("\t>>>ionizable test in full train\n\t\t", ionizable_test["Log_MP_RATIO"].isin(full_train["Log_MP_RATIO"]).value_counts("Log_MP_RATIO"))
print("\t>>>neutral test in full train\n\t\t", neutral_test["Log_MP_RATIO"].isin(full_train["Log_MP_RATIO"]).value_counts("Log_MP_RATIO"))
print("\t>>>full test in ionizable train\n\t\t", full_test["Log_MP_RATIO"].isin(ionizable_train["Log_MP_RATIO"]).value_counts("Log_MP_RATIO"))
print("\t>>>full test in ionizable train\n\t\t", full_test["Log_MP_RATIO"].isin(neutral_train["Log_MP_RATIO"]).value_counts("Log_MP_RATIO"))

===== FULL =====
	Train:	 (146, 358) 
	Test:	 (37, 358)
===== IONIZABLE =====
	Train:	 (58, 344) 
	Test:	 (14, 344)
===== NEUTRAL =====
	Train:	 (89, 332) 
	Test:	 (22, 332)
===== CHECK IF TESTS ARE IN OTHER DATAFRAMES =====
	>>>ionizable test in full train
		 Log_MP_RATIO
False    14
Name: count, dtype: int64
	>>>neutral test in full train
		 Log_MP_RATIO
False    22
Name: count, dtype: int64
	>>>full test in ionizable train
		 Log_MP_RATIO
False    37
Name: count, dtype: int64
	>>>full test in ionizable train
		 Log_MP_RATIO
False    37
Name: count, dtype: int64
===== FULL =====
	Train:	 (146, 358) 
	Test:	 (37, 358)
===== IONIZABLE =====
	Train:	 (58, 344) 
	Test:	 (14, 344)
===== NEUTRAL =====
	Train:	 (89, 332) 
	Test:	 (22, 332)
===== CHECK IF TESTS ARE IN OTHER DATAFRAMES =====
	>>>ionizable test in full train
		 Log_MP_RATIO
False    14
Name: count, dtype: int64
	>>>neutral test in full train
		 Log_MP_RATIO
False    22
Name: count, dtype: int64
	>>>full test in ionizable train

In [13]:
unfiltered_full_train = df_dict["full"].loc[df_dict["full"]["set"] == "TRAINING"].copy().drop(columns=["set"])
unfiltered_full_test = df_dict["full"].loc[df_dict["full"]["set"] == "TEST"].copy().drop(columns=["set"])

unfiltered_ionizable_train = df_dict["ionizable"].loc[df_dict["ionizable"]["set"] == "TRAINING"].copy().drop(columns=["set"])
unfiltered_ionizable_test = df_dict["ionizable"].loc[df_dict["ionizable"]["set"] == "TEST"].copy().drop(columns=["set"])

unfiltered_neutral_train = df_dict["neutral"].loc[df_dict["neutral"]["set"] == "TRAINING"].copy().drop(columns=["set"])
unfiltered_neutral_test = df_dict["neutral"].loc[df_dict["neutral"]["set"] == "TEST"].copy().drop(columns=["set"])


print("Shape of unfiltered:"
      "\n\tfull\n\t\tTrain\t:\t", unfiltered_full_train.shape,"\n\t\tTest\t:\t", unfiltered_full_test.shape,
      "\n\tionizable\n\t\tTrain\t:\t", unfiltered_ionizable_train.shape,"\n\t\tTest\t:\t", ionizable_test.shape,
      "\n\tneutral\n\t\tTrain\t:\t", neutral_train.shape,"\n\t\tTest\t:\t", neutral_test.shape,
      )

print("===== CHECK IF UNFILTERED TESTS ARE IN OTHER DATAFRAMES =====")
print("\t>>>ionizable test in full train\n\t\t", unfiltered_ionizable_test["Log_MP_RATIO"].isin(unfiltered_full_train["Log_MP_RATIO"]).value_counts("Log_MP_RATIO"))
print("\t>>>neutral test in full train\n\t\t", unfiltered_neutral_test["Log_MP_RATIO"].isin(unfiltered_neutral_train["Log_MP_RATIO"]).value_counts("Log_MP_RATIO"))
print("\t>>>full test in ionizable train\n\t\t", unfiltered_full_test["Log_MP_RATIO"].isin(unfiltered_full_train["Log_MP_RATIO"]).value_counts("Log_MP_RATIO"))
print("\t>>>full test in ionizable train\n\t\t", unfiltered_full_test["Log_MP_RATIO"].isin(unfiltered_full_train["Log_MP_RATIO"]).value_counts("Log_MP_RATIO"))

with pd.ExcelWriter("unfiltered_data.xlsx") as writer:
    unfiltered_full_train.to_excel(writer, sheet_name="full_train", index=False)
    unfiltered_full_test.to_excel(writer, sheet_name="full_test", index=False)

    unfiltered_ionizable_train.to_excel(writer, sheet_name="ionizable_train", index=False)
    unfiltered_ionizable_test.to_excel(writer, sheet_name="ionizable_test", index=False)

    unfiltered_neutral_train.to_excel(writer, sheet_name="neutral_train", index=False)
    unfiltered_neutral_test.to_excel(writer, sheet_name="neutral_test", index=False)


with pd.ExcelWriter("filtered_data.xlsx") as writer:
    full_train.to_excel(writer, sheet_name="full_train", index=False)
    full_test.to_excel(writer, sheet_name="full_test", index=False)

    ionizable_train.to_excel(writer, sheet_name="ionizable_train", index=False)
    ionizable_test.to_excel(writer, sheet_name="ionizable_test", index=False)

    neutral_train.to_excel(writer, sheet_name="neutral_train", index=False)
    neutral_test.to_excel(writer, sheet_name="neutral_test", index=False)



Shape of unfiltered:
	full
		Train	:	 (146, 479) 
		Test	:	 (37, 479) 
	ionizable
		Train	:	 (58, 479) 
		Test	:	 (14, 344) 
	neutral
		Train	:	 (89, 332) 
		Test	:	 (22, 332)
===== CHECK IF UNFILTERED TESTS ARE IN OTHER DATAFRAMES =====
	>>>ionizable test in full train
		 Log_MP_RATIO
True    1.0
Name: proportion, dtype: float64
	>>>neutral test in full train
		 Log_MP_RATIO
False    1.0
Name: proportion, dtype: float64
	>>>full test in ionizable train
		 Log_MP_RATIO
False    0.972973
True     0.027027
Name: proportion, dtype: float64
	>>>full test in ionizable train
		 Log_MP_RATIO
False    0.972973
True     0.027027
Name: proportion, dtype: float64
Shape of unfiltered:
	full
		Train	:	 (146, 479) 
		Test	:	 (37, 479) 
	ionizable
		Train	:	 (58, 479) 
		Test	:	 (14, 344) 
	neutral
		Train	:	 (89, 332) 
		Test	:	 (22, 332)
===== CHECK IF UNFILTERED TESTS ARE IN OTHER DATAFRAMES =====
	>>>ionizable test in full train
		 Log_MP_RATIO
True    1.0
Name: proportion, dtype: float64
	>>>neu

In [14]:
from sklearn.preprocessing import StandardScaler


def Corr_list_with_output(df, target=0.5):
    cor = df.corr()      # Using Pearson Correlation
    cor_target = abs(cor['Log_MP_RATIO'])
    #Selecting highly correlated features
    features = cor_target[cor_target>target]
    return features

# unfiltered_full_train = pd.DataFrame(StandardScaler().fit_transform(unfiltered_full_train), columns = unfiltered_full_train.columns)
unfiltered_full_train
# # y = full_train.corrwith(full_train["Log_MP_RATIO"], method="pearson").abs()
# y = Corr_list_with_output(unfiltered_full_train, target=0.5)
# # correlated_to_y = full_train[y.index]
# # correlated_to_y
# y

,Log_MP_RATIO,apol,ASA,ASA+,ASA-,ASA_H,ASA_P,a_acc,a_acid,a_aro,...,vsurf_Wp3,vsurf_Wp4,vsurf_Wp5,vsurf_Wp6,vsurf_Wp7,vsurf_Wp8,Weight,weinerPath,weinerPol,zagreb
0,-0.119186,22.785137,341.43088,196.220180,145.210690,239.01727,102.413610,2,0,6,...,64.000,22.375,9.625,3.000,0.000,0.0,151.16499,166,11,50
1,0.363698,43.759998,488.54459,0.000000,488.544590,0.00000,488.544590,0,0,0,...,0.250,0.000,0.000,0.000,0.000,0.0,545.54596,680,78,172
2,0.698970,30.486586,498.01099,58.344425,439.666560,0.00000,498.010990,2,0,0,...,80.625,33.000,13.750,4.625,0.125,0.0,499.14099,1864,72,164
3,0.761176,38.435482,528.63080,414.456510,114.174280,291.23990,237.390880,2,0,5,...,190.000,53.375,19.625,5.000,0.375,0.0,253.35400,664,18,74
4,-0.050610,8.322758,197.57458,106.382390,91.192207,133.01698,64.557610,1,0,0,...,19.250,6.250,2.750,0.750,0.000,0.0,46.06900,4,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,0.606785,37.137585,508.13315,108.890020,399.243130,508.13315,0.000000,0,0,12,...,78.875,0.000,0.000,0.000,0.000,0.0,390.86398,732,39,112
142,0.754503,35.624378,490.22906,120.277710,369.951350,490.22906,0.000000,0,0,12,...,63.250,0.000,0.000,0.000,0.000,0.0,356.41898,648,35,106
143,0.553557,37.137585,502.72696,108.893750,393.833220,502.72696,0.000000,0,0,12,...,70.250,0.000,0.000,0.000,0.000,0.0,390.86398,731,39,112
144,0.622817,34.111172,469.09003,129.427840,339.662200,469.09003,0.000000,0,0,12,...,48.625,0.000,0.000,0.000,0.000,0.0,321.97400,571,31,100


,Log_MP_RATIO,apol,ASA,ASA+,ASA-,ASA_H,ASA_P,a_acc,a_acid,a_aro,...,vsurf_Wp3,vsurf_Wp4,vsurf_Wp5,vsurf_Wp6,vsurf_Wp7,vsurf_Wp8,Weight,weinerPath,weinerPol,zagreb
0,-0.119186,22.785137,341.43088,196.220180,145.210690,239.01727,102.413610,2,0,6,...,64.000,22.375,9.625,3.000,0.000,0.0,151.16499,166,11,50
1,0.363698,43.759998,488.54459,0.000000,488.544590,0.00000,488.544590,0,0,0,...,0.250,0.000,0.000,0.000,0.000,0.0,545.54596,680,78,172
2,0.698970,30.486586,498.01099,58.344425,439.666560,0.00000,498.010990,2,0,0,...,80.625,33.000,13.750,4.625,0.125,0.0,499.14099,1864,72,164
3,0.761176,38.435482,528.63080,414.456510,114.174280,291.23990,237.390880,2,0,5,...,190.000,53.375,19.625,5.000,0.375,0.0,253.35400,664,18,74
4,-0.050610,8.322758,197.57458,106.382390,91.192207,133.01698,64.557610,1,0,0,...,19.250,6.250,2.750,0.750,0.000,0.0,46.06900,4,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,0.606785,37.137585,508.13315,108.890020,399.243130,508.13315,0.000000,0,0,12,...,78.875,0.000,0.000,0.000,0.000,0.0,390.86398,732,39,112
142,0.754503,35.624378,490.22906,120.277710,369.951350,490.22906,0.000000,0,0,12,...,63.250,0.000,0.000,0.000,0.000,0.0,356.41898,648,35,106
143,0.553557,37.137585,502.72696,108.893750,393.833220,502.72696,0.000000,0,0,12,...,70.250,0.000,0.000,0.000,0.000,0.0,390.86398,731,39,112
144,0.622817,34.111172,469.09003,129.427840,339.662200,469.09003,0.000000,0,0,12,...,48.625,0.000,0.000,0.000,0.000,0.0,321.97400,571,31,100
